### PLAN
1. Load up to Jun 2022 into BigQuery
2. Create github action to run BQ insert script nightly
3. Set grouped database table (history_df) in BigQuery to refresh nightly after insert
4. Make script to pull data history_df from BigQuery data

8. For streamlit app append [history_df, current_df, future_df] to make the combo_df
    - put history_df into cache using st.experimental_memo so its only called once each time app is opened

In [1]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import ab_power_trader
import json
import http.client
import certifi
import ssl
import os

In [ ]:
# Pull current day of data (current_df)
from datetime import datetime, timedelta
import pandas as pd
import pull_nrg_data
from google.oauth2 import service_account
from pandasql import sqldf

def get_streams():
    streams = pd.read_csv('stream_codes.csv')
    lst = [int(id) for id in streams[(streams['timeInterval']=='1 hr') & (streams['intervalType']=='supply')]['streamId']]
    return lst
[86, 322684, 322677, 87, 85, 23695, 322665, 23694]
# Path to Google auth credentials
#credentials = service_account.Credentials.from_service_account_info(st.secrets["gcp_service_account"])

if __name__ == '__main__':
    streamIds = [86, 322684, 322677, 87, 85, 23695, 322665, 23694]
    current_df = pd.DataFrame([])
    today = datetime.now()
    for id in streamIds:
        accessToken, tokenExpiry = pull_nrg_data.getToken()
        try:
            APIdata = pull_nrg_data.pull_data(today.strftime('%m/%d/%Y'), today.strftime('%m/%d/%Y'), id, accessToken, tokenExpiry)
            pull_nrg_data.release_token(accessToken)
            APIdata['timeStamp'] = pd.to_datetime(APIdata['timeStamp'])
            current_df = pd.concat([current_df, APIdata], axis=0)
        except:
            pull_nrg_data.release_token(accessToken)
            pass
    query = '''
        SELECT  
                fuelType,
                strftime('%Y', timeStamp) AS year,
                strftime('%m', timeStamp) AS month,
                strftime('%d', timeStamp) AS day,
                strftime('%H', timeStamp) AS hour,
                avg(value) 
        FROM current_df
        GROUP BY fuelType, year, month, day, hour
        '''
sqldf(query, globals())

In [ ]:
pull_nrg_data.release_token(accessToken)

In [2]:
# Release token
import certifi
import ssl
import http.client
server = 'api.nrgstream.com'

def release_token(accessToken):
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

accessToken = 'x6UHxLeva7VIEVeE-JSTuFLe-sQPxkNe2bcdy6agGFYFewPgTUNO7jyHq3hfg4SjFewtFMXHiVwvcWKg4UUS0---J9fEVkm5Xx3uRDG_e2P1xrX2rABU-Nc8HK4bVFJTjd9keD5u75HZUZiP6mM8sReb91F7xqXubbvPDUOGiHGUIx3iVPX-YGTDuOaO4DSOe44E-IwWVU-fPXeuNwmlgaajjl78NxreNXh6b_d6ZZYpsgKmvES1wl8QQNIzLWFuejVjAjYo-p4Up2LtYijV3G7ouLe1jJ7hV89eQdCsT2We4swCkBQDyywXQcqzVsmgfzGADi2gOKB9KyFCr7HHvuu04eY'
release_token(accessToken)

token released


In [ ]:
# Pull data from BQ
import os
import pandas as pd
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# Path to Google auth credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ryan-bulger/power-trader/google-big-query.json'
sql = "SELECT * FROM nrgdata.grouped_data"
df = bigquery.Client().query(sql).to_dataframe()

In [ ]:
def stream_data(streamIds, streamNames, years):
    stream_df = pd.DataFrame([])
    for id in streamIds:
        server = 'api.nrgstream.com'
        year_df = pd.DataFrame([])
        for yr in years:
            accessToken, tokenExpiry = pull_nrg_data.getToken()
            # Define start & end dates
            startDate = date(yr,1,1).strftime('%m/%d/%Y')
            endDate = date(yr,12,31).strftime('%m/%d/%Y')
            # NRG API connection
            path = f'/api/StreamData/{id}?fromDate={startDate}&toDate={endDate}'
            headers = {'Accept': 'Application/json', 'Authorization': f'Bearer {accessToken}'}
            context = ssl.create_default_context(cafile=certifi.where())
            conn = http.client.HTTPSConnection(server, context=context)
            conn.request('GET', path, None, headers)
            res = conn.getresponse()
            # Load json data from API & create pandas df
            jsonData = json.loads(res.read().decode('utf-8'))
            df = pd.json_normalize(jsonData, record_path='data')
            # Close NRG API connection
            conn.close()
            # Concat years for each stream
            year_df = pd.concat([year_df,df], axis=0)
            # Release NRG API access token
            pull_nrg_data.release_token(accessToken)
        # Rename year_df cols
        year_df.rename(columns={0:'timeStamp', 1:f'{streamNames[id]}'}, inplace=True)
        print(year_df)
        # Change timeStamp to datetime
        year_df['timeStamp'] = pd.to_datetime(year_df['timeStamp'])
        # Re-index the year_df
        year_df.set_index('timeStamp', inplace=True)
        # Join year_df to outages dataframe
        stream_df = pd.concat([stream_df,year_df], axis=1, join='outer')
    return stream_df

streamIds = [44648, 118361, 322689, 118362, 147262, 322675, 322682, 44651]
#streamIds = [44648]
streamNames = {44648:'Coal', 118361:'Gas', 322689:'Dual Fuel', 118362:'Hydro', 147262:'Wind', 322675:'Solar', 322682:'Energy Storage', 44651:'Biomass & Other'}
years = [datetime.now().year, datetime.now().year+1, datetime.now().year+2]
outage_df = stream_data(streamIds, streamNames, years)
#print(outage_df)
# Reset index so dataframe can be plotted with Altair
outage_df.reset_index(inplace=True)
outage_df = pd.melt(outage_df, 
                id_vars=['timeStamp'],
                value_vars=['Coal', 'Gas', 'Dual Fuel', 'Hydro', 'Wind', 'Solar', 'Energy Storage', 'Biomass & Other'],
                var_name='Source',
                value_name='Value')
outage_df

In [ ]:
# Joining daily to hourly data
import pandas as pd

intraday = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='H'),data=[i for i in range(145)], columns=['hourly'])
daily = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='D'), data=[i for i in range(7)], columns=['daily'])
df = intraday.join(daily).fillna(method='ffill')
df

In [3]:
import ab_power_trader
outage_df = ab_power_trader.outages().astype('int32')
outage_df

2022-08-09 08:46:28.290 
  command:

    streamlit run /home/ryan-bulger/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


,Coal,Natural Gas,Dual Fuel,Hydro,Wind,Solar,Energy Storage,Biomass & Other
timeStamp,,,,,,,,
2022-01-01 00:00:00-07:00,700,740,0,120,130,400,0,50
2022-02-01 00:00:00-07:00,0,820,0,140,210,400,0,60
2022-03-01 00:00:00-07:00,80,990,0,120,120,60,0,50
2022-04-01 00:00:00-06:00,310,1760,0,140,50,450,0,120
2022-05-01 00:00:00-06:00,0,1740,0,150,10,390,0,120
2022-06-01 00:00:00-06:00,0,1530,0,70,10,330,0,110
2022-07-01 00:00:00-06:00,0,1130,0,30,30,220,0,100
2022-08-01 00:00:00-06:00,20,1150,0,50,40,180,0,80
2022-09-01 00:00:00-06:00,0,1440,0,80,0,130,0,50


In [21]:
old_outage_df = ab_power_trader.outages().astype('int32').reset_index()
old_outage_df['Hydro'] = [x - random.randint(0,51) if x >=50 else x + random.randint(0,51)for x in old_outage_df['Hydro']]
old_outage_df['Solar'] = [x - random.randint(0,51) if x >=100 else x + random.randint(0,51) for x in old_outage_df['Solar']]
old_outage_df['Natural Gas'] = [x - random.randint(0,51) if x >=500 else x + random.randint(0,51) for x in old_outage_df['Natural Gas']]
old_outage_df['Coal'] = [x - random.randint(0,51) if x >=50 else x + random.randint(0,51) for x in old_outage_df['Coal']]
old_outage_df.set_index(['timeStamp'], inplace=True)
old_outage_df

,Coal,Natural Gas,Dual Fuel,Hydro,Wind,Solar,Energy Storage,Biomass & Other
timeStamp,,,,,,,,
2022-01-01 00:00:00-07:00,687,727,0,103,130,364,0,50
2022-02-01 00:00:00-07:00,21,787,0,134,210,369,0,60
2022-03-01 00:00:00-07:00,67,971,0,112,120,86,0,50
2022-04-01 00:00:00-06:00,300,1749,0,137,50,402,0,120
2022-05-01 00:00:00-06:00,30,1733,0,148,10,371,0,120
2022-06-01 00:00:00-06:00,45,1515,0,56,10,310,0,110
2022-07-01 00:00:00-06:00,34,1110,0,55,30,205,0,100
2022-08-01 00:00:00-06:00,65,1115,0,0,40,143,0,80
2022-09-01 00:00:00-06:00,4,1440,0,55,0,114,0,50


In [84]:
streamIds = [86, 322684, 322677, 87, 85, 23695, 322665, 23694]
current_df = pd.DataFrame([])
today = datetime.now()
for id in streamIds:
    accessToken, tokenExpiry = pull_nrg_data.getToken()
    try:
        APIdata = pull_nrg_data.pull_data(today.strftime('%m/%d/%Y'), today.strftime('%m/%d/%Y'), id, accessToken, tokenExpiry)
        pull_nrg_data.release_token(accessToken)
        APIdata['timeStamp'] = pd.to_datetime(APIdata['timeStamp'])
        current_df = pd.concat([current_df, APIdata], axis=0)
    except:
        pull_nrg_data.release_token(accessToken)
        pass
current_df

,timeStamp,value,streamId,assetCode,streamName,fuelType,subfuelType,timeInterval,intervalType
0,2022-08-09 00:00:00,835,86,NaN,Coal,Coal,Coal,5 min,supply
1,2022-08-09 00:05:00,838,86,NaN,Coal,Coal,Coal,5 min,supply
2,2022-08-09 00:10:00,839,86,NaN,Coal,Coal,Coal,5 min,supply
3,2022-08-09 00:15:00,838,86,NaN,Coal,Coal,Coal,5 min,supply
4,2022-08-09 00:20:00,840,86,NaN,Coal,Coal,Coal,5 min,supply
...,...,...,...,...,...,...,...,...,...
187,2022-08-09 15:35:00,202,23694,NaN,Wind,Wind,Wind,5 min,supply
188,2022-08-09 15:40:00,206,23694,NaN,Wind,Wind,Wind,5 min,supply
189,2022-08-09 15:45:00,201,23694,NaN,Wind,Wind,Wind,5 min,supply
190,2022-08-09 15:50:00,205,23694,NaN,Wind,Wind,Wind,5 min,supply


In [100]:
query = '''
SELECT
    timeStamp,
    AVG(value) AS value,
    fuelType,
    strftime('%Y', timeStamp) AS year,
    strftime('%m', timeStamp) AS month,
    strftime('%d', timeStamp) AS day,
    strftime('%H', timeStamp) AS hour
FROM current_df
WHERE value > 0
GROUP BY fuelType, year, month, day, hour
'''
non_zero_df = sqldf(query, locals())
non_zero_df

,timeStamp,value,fuelType,year,month,day,hour
0,2022-08-09 00:00:00.000000,250.583333,Biomass & Other,2022,08,09,00
1,2022-08-09 01:00:00.000000,249.083333,Biomass & Other,2022,08,09,01
2,2022-08-09 02:00:00.000000,241.166667,Biomass & Other,2022,08,09,02
3,2022-08-09 03:00:00.000000,234.916667,Biomass & Other,2022,08,09,03
4,2022-08-09 04:00:00.000000,247.583333,Biomass & Other,2022,08,09,04
...,...,...,...,...,...,...,...
85,2022-08-09 11:00:00.000000,110.666667,Wind,2022,08,09,11
86,2022-08-09 12:00:00.000000,157.083333,Wind,2022,08,09,12
87,2022-08-09 13:00:00.000000,203.333333,Wind,2022,08,09,13
88,2022-08-09 14:00:00.000000,211.500000,Wind,2022,08,09,14


In [104]:
query = '''
SELECT
    timeStamp,
    avg(value) AS value,
    fuelType,
    strftime('%Y', timeStamp) AS year,
    strftime('%m', timeStamp) AS month,
    strftime('%d', timeStamp) AS day,
    strftime('%H', timeStamp) AS hour
FROM current_df
WHERE fuelType NOT IN (
    SELECT
    fuelType
    FROM current_df
    WHERE value > 0
    GROUP BY fuelType
)
GROUP BY fuelType, year, month, day, hour
'''
zero_df = sqldf(query, locals())
zero_df

,timeStamp,value,fuelType,year,month,day,hour
0,2022-08-09 00:00:00.000000,0.0,Dual Fuel,2022,08,09,00
1,2022-08-09 01:00:00.000000,0.0,Dual Fuel,2022,08,09,01
2,2022-08-09 02:00:00.000000,0.0,Dual Fuel,2022,08,09,02
3,2022-08-09 03:00:00.000000,0.0,Dual Fuel,2022,08,09,03
4,2022-08-09 04:00:00.000000,0.0,Dual Fuel,2022,08,09,04
5,2022-08-09 05:00:00.000000,0.0,Dual Fuel,2022,08,09,05
6,2022-08-09 06:00:00.000000,0.0,Dual Fuel,2022,08,09,06
7,2022-08-09 07:00:00.000000,0.0,Dual Fuel,2022,08,09,07
8,2022-08-09 08:00:00.000000,0.0,Dual Fuel,2022,08,09,08
9,2022-08-09 09:00:00.000000,0.0,Dual Fuel,2022,08,09,09


In [105]:
df = pd.concat([non_zero_df,zero_df], axis=0)
df

,timeStamp,value,fuelType,year,month,day,hour
0,2022-08-09 00:00:00.000000,250.583333,Biomass & Other,2022,08,09,00
1,2022-08-09 01:00:00.000000,249.083333,Biomass & Other,2022,08,09,01
2,2022-08-09 02:00:00.000000,241.166667,Biomass & Other,2022,08,09,02
3,2022-08-09 03:00:00.000000,234.916667,Biomass & Other,2022,08,09,03
4,2022-08-09 04:00:00.000000,247.583333,Biomass & Other,2022,08,09,04
...,...,...,...,...,...,...,...
27,2022-08-09 11:00:00.000000,0.000000,Energy Storage,2022,08,09,11
28,2022-08-09 12:00:00.000000,0.000000,Energy Storage,2022,08,09,12
29,2022-08-09 13:00:00.000000,0.000000,Energy Storage,2022,08,09,13
30,2022-08-09 14:00:00.000000,0.000000,Energy Storage,2022,08,09,14


In [20]:
tE = datetime(2022,8,12,10,48)

def te():
    print(tE)
    if tE < datetime.now():
        print('reset')
        tE = datetime.now()
        print(tE)

te()

UnboundLocalError: local variable 'tE' referenced before assignment